<a href="https://colab.research.google.com/github/veroorli/ProjetProg/blob/master/dm1bdle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Detail sur les personnes et les oeuvres

#### TitleDetail

In [ ]:
%%sql
create or replace temp view TitleDetail as
select t.id, t.title, t.production_year, k.kind, it.info as property, m.info as value
from Title t, Movie_Info m, Info_type it, Kind_Type k
where t.id = m.movie_id 
and m.info_type_id = it.id
and t.kind_id = k.id
;

select * 
from TitleDetail t
order by  t.production_year desc, t.title
limit 100;

,id,title,production_year,kind,property,value
0,2310404,Star Trek 3,2016,movie,genres,Sci-Fi
1,2310404,Star Trek 3,2016,movie,release dates,USA:2016
2,2310404,Star Trek 3,2016,movie,genres,Adventure
3,2310404,Star Trek 3,2016,movie,genres,Action
4,2310404,Star Trek 3,2016,movie,color info,Color


Mémoriser le résultat de TitleDetail pour éviter de le calculer plusieurs fois

In [ ]:
%%sql
cache table TitleDetail

""


#### NameDetail

In [ ]:
%%sql
create or replace temp view NameDetail as
select n.id, n.name, n.gender, i.info as property, p.info as value
from Name n, Person_Info p, Info_type i
where n.id = p.person_id
and p.info_type_id = i.id
;
 
select * 
from NameDetail n
where n.name like 'Smi%'
order by n.name, n.property
limit 100;

,id,name,gender,property,value
0,2544489,"Smid, Coosje",f,trivia,Daughter of 'Ernst Daniël Smid' (qv) and Rosem...
1,1454381,"Smid, Ernst Daniël",m,birth date,6 May 1953
2,1454381,"Smid, Ernst Daniël",m,birth notes,"Enschede, Overijssel, Netherlands"
3,1454381,"Smid, Ernst Daniël",m,magazine cover photo,"""Ncrv Gids"" (Netherlands), 17 April 1999, Iss. 16"
4,1454381,"Smid, Ernst Daniël",m,other works,Provided the voice of Ocious P. Potter in the ...


Mémoriser le résultat de NameDetail pour éviter de le calculer plusieurs fois

In [ ]:
%%sql
cache table NameDetail;

""


## QUESTIONS

### Exercice 1: Extraction d'information

Définir ces relations apportant des informations structurées sur les personnes. Le schéma de chaque table est donné à titre indicatif et vous pouvez l'adapter et le compléter si nécessaire.

#### Question 1 : Salary
 **Salary** (id, name, titleId, titleName, salary, extra, currency) avec
  - *salary* est le salaire perçu par une personne pour un film
  - *frequency* indique si le salaire est pour le film, un épisode de la série, une semaine de tournage, etc...
  - *extra* est un complément de salaire (exple pourcentage sur le revenu du film)
  - *currency* est la devise du salaire
 

Voir la propriété *salary history*. Le champ *titleId* peut être nul si aucune correspondance n'est trouvée dans la table TitleDetail. Le cas échéant, essayer de trouver une correspondance avec le fichier *title_all* contenant tous les films.

In [ ]:
%%sql

-- Tous les regex ont été construits et améliorés au fur et à mesure que l'on trouve des formats différents présents dans les textes.

-- Extraction du titre et de l'année de production des personnes de NameDetail pour préparer la jointure
create or replace temp view T as
select id as nameId, name, value as nameValue, 
trim(regexp_extract(value, '_"?([^\\(\\"]+)"? \\(', 1)) as titleName, 
regexp_extract(value, '_([^\\(]+) \\(([0-9]*).*\\)_', 2) as production_year
from NameDetail
where property='salary history';

-- Création de la table Salaire
create or replace temp view Salary as
select distinct nameId, name, nameValue,

-- Condition : si le champ titleId n'est pas nul, alors on l'affiche, sinon on cherche une correspondance dans title_all
if (id is not null, id,
  (select first(ta.id) from TitleAll ta where T.titleName = ta.title and T.production_year = ta.production_year)
) as titleId,

-- Nom du titre : On extrait les titres qui sont entre '_' sans les dates.
titleName,

-- Salaire : On extrait le montant qui se situe après les doubles ':'
-- Le salaire peut contenir des chiffres, des virgules ou un 'm'
regexp_extract(nameValue, '[^:]+::([^0-9]*)([0-9,m]+)', 2) as salary,

-- Fréquence :  Les fréquences sont situées après le salaire, suivi d'un '/' ou le mot 'per'
regexp_extract(nameValue, '[^:]+::([^0-9]*)([0-9,m]+).*(/|per )([^ +]+)*', 4) as frequency,

-- Devise : La condition if sert à vérifier si la devise se trouve avant ou après le salaire.
-- Si les deux ':' ne sont suivis par un chiffre, alors on extrait la devise qui est après salaire, sinon elle se situe avant le nombre.
if(regexp_extract(nameValue, '[^:]+::([^0-9]*)([0-9,m]+)', 1)='',
    regexp_extract(nameValue, '[^:]+::([^0-9]*)([0-9,m]+) ([^ +]+)', 3), 
    regexp_extract(nameValue, '[^:]+::([^0-9]*)(([0-9,]+|(m ))*)', 1)) as currency,

-- Extra : On extrait le texte qui suit '+' ou 'plus'
regexp_extract(nameValue, '[^:]+::([^0-9]*)([0-9,m]+)(([^+]*\\+)|(.*plus))([^\\)]*)', 6) as extra

from T
-- Jointure à gauche sur NameDetail avec le titre et la date de production
left join TitleDetail on T.titleName = TitleDetail.title and T.production_year = TitleDetail.production_year;

select * from Salary;

,nameId,name,nameValue,titleId,titleName,salary,frequency,currency,extra
0,2037716,"Garland, Judy","_A Star Is Born (1954)_ (qv)::$100,000 + 50% o...",854543,A Star Is Born,"100,000",,$,50% of profits
1,2437832,"Pulver, Liselotte",_A Time to Love and a Time to Die (1958)_ (qv)...,1658907,A Time to Love and a Time to Die,50,,$,
2,1984724,"Elfman, Jenna","_""Accidentally on Purpose"" (2009)_ (qv)::$125,...",37156,Accidentally on Purpose,"125,000",episode,$,
3,1094728,"Muniz, Frankie","_Agent Cody Banks (2003)_ (qv)::$2,000,000",1669115,Agent Cody Banks,"2,000,000",,$,
4,112128,"Beatty, Warren","_All Fall Down (1962)_ (qv)::$60,000",1675454,All Fall Down,"60,000",,$,


#### Question 2 : Height
 **Height**(id, name, heightInCm) qui recense la taille des personnes de manière exploitable, en les reconvertissant en tailles en cm, alors que les entrées de la base sont  en plusieurs unités.

In [ ]:
%%sql

create or replace temp view Height as
select id, name, value,

-- Les valeurs de la propriété height peuvent se trouver dans 3 formats différents : en cm, en feet ou en feet+inches.
-- On teste dans un premier temps si la valeur est déjà en centimètre (cm)
if (locate('cm', value)>0, regexp_extract(value, "(.+) cm", 1),

-- Si la première condition n'est pas vérifiée, on teste si la valeur contient ou pas des inches
-- On convertit les feets en centimètres si il n'y a pas de inches
if (locate('"', value)=0 , cast(regexp_extract(value, "(.+)'", 1) as int)*30.48,

-- Si il y a des 1/2 inches, on convertit les inches en feet en rajoutant 1/2 aux inches, puis on les met en centimètres
if (locate('1/2', value)>0, (cast(regexp_extract(value, "(.+)' ([0-9]+)", 1) as int)+ (cast(regexp_extract(value, "(.+)' ([0-9]+)", 2) as int)+0.5)/12)*30.48,

-- Sinon, on transforme les inches en feets et on les convertit ensuite en centimètres
(cast(regexp_extract(value, "(.+)' ?([0-9]+)", 1) as int)+ cast(regexp_extract(value, "(.+)' ?([0-9]+)", 2) as int)/12)*30.48))) as height_in_cm

from NameDetail
where property = 'height'
;

select id, name, value as height, height_in_cm as heightInCm
from Height
order by float(height_in_cm) desc

,id,name,height,heightInCm
0,111308,"Bear, Bart the","9' 6""",289.56
1,413534,"Douglass, Jack","8' 11 1/2""",273.04998984
2,1016803,"McGrory, Matthew","7' 6""",228.6
3,576560,"Gonzáles, Jorge","7' 6""",228.6
4,1711328,"Yao, Ming","7' 5 1/2""",227.32998984


#### Question 3 : Magazine

 **MagazineCover**(id, name, magazine, country, day, month, year) donnant le nom du magazine dans lequel la personne est sur la couverture, avec le pays du magazine et la date de parution. Voir la propriété *magazine cover photo*.

In [ ]:
%%sql

create or replace temp view MagazineCover as
select id, name, --value,

-- Magazine : On extrait ce qui est entre les guillemets
regexp_extract(value, '\\"(.+)\\"( \\(([^\\)]*)\\))*', 1) as Cover_from_Magazine,

-- Pays : Le pays se trouve entre parenthèses après le magazine
regexp_extract(value, '\\"(.+)\\"( \\(([^\\)]*)\\))*', 3) as Country,

-- Date : Le jour peut ne pas apparaître, on extrait un ou deux chiffres consécutifs
regexp_extract(value, '\\"(.+)\\"( \\(([^\\)]*)\\))*, (Vol\\. [0-9]+, )?(Iss\\. [0-9]+, )?([0-9]{1,2}) ', 6) as Day,

-- Mois : Le mois peut ne pas apparaître, on extrait les lettres qui se trouvent entre les nombres
regexp_extract(value, '\\"(.+)\\"( \\(([^\\)]*)\\))*, (Vol\\. [0-9]+, )?(Iss\\. [0-9]+, )?[^a-zA-Z]*([a-zA-Z\\-/]*)( [0-9]|$|,)', 6) as Month,

-- Année : L'année peut ne pas apparaître, la base contient que des années de 4 chiffres consécutifs, on extrait alors suivant cette observation.
regexp_extract(value, '\\"(.+)\\"( \\(([^\\)]*)\\))*, (Vol\\. [0-9]+, )?(Iss\\. [0-9]+, )?[^a-zA-Z]{0,2}[a-zA-Z \\-/]*[^a-zA-Z]{0,2}(, )?([0-9]{4})', 7) as Year

from NameDetail
where property = 'magazine cover photo';

select * from MagazineCover;

,id,name,Cover_from_Magazine,Country,Day,Month,Year
0,4662,"Abramoff, Jack",Time,USA,16,January,2006
1,21326,"Albers, Hans",Der Spiegel,Germany,14,February,2000
2,21326,"Albers, Hans",Grosse Österreich Illustrierte,Austria,25,July,1953
3,21326,"Albers, Hans",Das Ufer,West Germany,,April,1951
4,21326,"Albers, Hans",Quick,West Germany,25,February,1951


#### Question 4 : Trivia

Deux tables détaillant les anecdotes connues sur des personnes, voir les valeurs de la propriété *trivia* d'une personne : 
- **TitleTrivia** (fromPersonId, fromPersonName, toTitleId, toTitleName, triviaText)
- **NameTrivia** (fromPersonId,  fromPersonName, toPersonId, toPersonName, triviaText)
    
Indications :  

Pour le champ toTitleId dans TitleTrivia : si le film n'est pas référencé dans la table TitleDetail, essayer de trouver une correspondance dans le fichier contenant tous les films (title_full).

Pour le champ toPersonId dans NameTrivia : si la personne n'est pas référencée dans la table NameDetail, essayer de trouver une correspondance dans le fichier contenant toutes les personnes (name_full).

In [ ]:
%%sql

create or replace temp view T_TitleTrivia as
select t.id, t.title,

-- On fait une sous-requête pour extraire les id des titres présents dans TitleDetail
(select first(t2.id) from TitleDetail t2 where t2.title = trim(regexp_extract(t.value, "_([^_]+)_ \\(qv\\)", 1))) as toTitleId,

-- On extrait le nom du premier titre mentionné dans le trivia : entre '_' et avant (qv)
trim(regexp_extract(t.value, "_([^_]+)_ \\(qv\\)", 1)) as toTitleName,
t.value as triviaText
from TitleDetail t
-- On recupère les trivias concernant les films : qv
where t.property = 'trivia' and t.value like '%(qv)%';

create or replace temp view TitleTrivia as
select t.id as fromTitleId, t.title as fromTitleName,

-- si l'id n'a pas été trouvé dans TitleDetail, alors on cherche une correspondance dans TitleAll
if (t.toTitleId is not null, t.toTitleId, 
(select first(ta.id) from TitleAll ta where t.toTitleName = ta.title)
) as toTitleId,

t.toTitleName, triviaText
from T_TitleTrivia t;


select * from TitleTrivia

,fromTitleId,fromTitleName,toTitleId,toTitleName,triviaText
0,1665420,Adam,NaN,"""America's Most Wanted"" (1988)",Based on the true story of the kidnapping of A...
1,669945,Jekyll,NaN,"""Doctor Who"" (2005)","'Michelle Ryan (II)' (qv), 'Meera Syal' (qv) a..."
2,438248,The Catwalk,NaN,"""Enterprise"" (2001)","This is the only time in the entire _""Enterpri..."
3,1417866,The Simpsons,NaN,"""Gunsmoke"" (1955)","With the show being renewed until 2011, it wil..."
4,843900,Cover Boy,NaN,"""He & She"" (1967)",The character of Ted Baxter was openly pattern...


In [ ]:
%%sql

create or replace temp view T_NameTrivia as
select n.id, n.name,

-- On fait une sous-requête pour extraire les id des personnes présentes dans NameDetail
(select first(n2.id) from NameDetail n2 where n2.name = trim(regexp_extract(n.value, "\\'([^\\']+)\\' \\(qv\\)", 1))) as toPersonId,

-- On extrait le nom de la première personne mentionnée dans le trivia : entre quote et avant (qv)
trim(regexp_extract(n.value, "\\'([^\\']+)\\' \\(qv\\)", 1)) as toPersonName,
n.value as triviaText
from NameDetail n
-- On recupère les trivias concernant les films : qv
where n.property = 'trivia' and n.value like '%(qv)%';


create or replace temp view NameTrivia as
select t.id as fromPersonId, t.name as fromPersonName,

-- si l'id n'a pas été trouvé dans NameDetail, alors on cherche une correspondance dans NameAll
if (t.toPersonId is not null, t.toPersonId, 
(select first(na.id) from NameAll na where t.toPersonName = na.name)
) as toPersonId,

t.toPersonName, triviaText
from T_NameTrivia t;


select * from NameTrivia

,fromPersonId,fromPersonName,toPersonId,toPersonName,triviaText
0,2555649,"Spears, Britney",113,*NSYNC,June 2001: Two Texas DJs created a panic for h...
1,104362,"Bass, Lance",113,*NSYNC,"Was the last member to join '*NSYNC' (qv), aft..."
2,104362,"Bass, Lance",113,*NSYNC,Performed with '*NSYNC' (qv) in the 1998 and 1...
3,104362,"Bass, Lance",113,*NSYNC,Member of boy group '*NSYNC' (qv).
4,104362,"Bass, Lance",113,*NSYNC,(November 1999) Performed with '*NSYNC' (qv) o...


#### Question : 5 Lien IMDB-YAGO

On suppose que certaines personnes sont présentes à la fois dans IMDB et dans YAGO mais elles ne sont pas structurées de la même façon.

5a) Compléter les Personnes existant dans NameDetail avec des informations venant de yago (voir **yagoPerson** et **actorFacts**). 
Pour cela, définir la relation qui donne le lieu de naissance et le conjoint (ou la conjointe) d'une personne :

  - **NameYago** (id, name, yagoName, yagoBirthPlace, yagoSpouse) 

5b) Compléter le contenu de NameYago pour les personnes de IMDB existant dans *name_full*.

In [ ]:
# Création des vues temporaires YagoPerson et ActorFacts pour la création des tables
yagoPerson.createOrReplaceTempView("YagoPerson")
actorFacts.createOrReplaceTempView("ActorFacts")

In [ ]:
%%sql
-- Création des tables de yagoPerson et actorFacts en retirant les patterns récurrents (les liens)

-- Création de la table YagoP à partir de yagoPerson
create or replace temp view YagoP as
select regexp_extract(subject, '<http://yago-knowledge.org/resource/([^>]*)>') as personName
from yagoPerson
;

-- Création de la table ActorF à partir de actorFacts
create or replace temp view ActorF as
select regexp_extract(subject, '<http://yago-knowledge.org/resource/([^>]*)>') as personName,
regexp_extract(property, '<http://schema.org/([^>]*)>') as property,
regexp_extract(object, '<http://yago-knowledge.org/resource/([^>]*)>') as value
from actorFacts
;

-- Affichage rapide des attributs des deux tables
select * 
from YagoP p, ActorF a 
where p.personName = a.personName

,personName,personName,property,value
0,50_Cent,50_Cent,award,Billboard_Music_Award
1,50_Cent,50_Cent,image,
2,50_Cent,50_Cent,nationality,United_States
3,50_Cent,50_Cent,familyName,Jackson_(name)
4,50_Cent,50_Cent,familyName,James_(surname)


In [ ]:
%%sql
-- 5a) Objectif : NameYago (id, name, yagoName, yagoBirthPlace, yagoSpouse)

create or replace temp view NameYago as
select distinct n.id, n.name, b.personName as yagoName, b.lieu as yagoBirthPlace, s.spouse as yagoSpouse
from NameDetail n,
-- Sous-requête pour obtenir le lieu de naissance
(select personName, value as lieu from ActorF where property = 'birthPlace') as b,
-- Sous-requête pour obtenir le nom du conjoint(e)
(select personName, value as spouse from ActorF where property = 'spouse') as s
-- Jointure sur les noms des personnes
where n.name = b.personName and n.name = s.personName
;

select * from NameYago

,id,name,yagoName,yagoBirthPlace,yagoSpouse
0,716114,Ice-T,Ice-T,"Newark,_New_Jersey",Coco_Austin
1,2208688,Kyla,Kyla,"Tondo,_Manila",Rich_Alvarez
2,2298105,Maïwenn,Maïwenn,Les_Lilas,Luc_Besson
3,2524210,Shabnam,Shabnam,Dhaka,Robin_Ghosh
4,2524329,Shadia,Shadia,Cairo,Emad_Hamdy


In [ ]:
%%sql
-- 5b)

create or replace temp view NameYagob as
select distinct n.id, n.name, b.personName as yagoName, b.lieu as yagoBirthPlace, s.spouse as yagoSpouse
from NameAll n,
-- Sous-requête pour obtenir le lieu de naissance
(select personName, value as lieu from ActorF where property = 'birthPlace') as b,
-- Sous-requête pour obtenir le nom du conjoint(e)
(select personName, value as spouse from ActorF where property = 'spouse') as s
-- Jointure sur les noms des personnes
where n.name = b.personName and n.name = s.personName
;

select * from NameYagob

,id,name,yagoName,yagoBirthPlace,yagoSpouse
0,442402,Eita,Eita,Niigata_Prefecture,Kaela_Kimura
1,1997731,Fairuz,Fairuz,Beirut,Assi_Rahbani
2,3251095,Fairuz,Fairuz,Beirut,Assi_Rahbani
3,3687536,Fairuz,Fairuz,Beirut,Assi_Rahbani
4,716114,Ice-T,Ice-T,"Newark,_New_Jersey",Coco_Austin


### Exercice 2 : Analyse

#### Question 6 : Restructuration
L'objectif de cette question est de savoir définir une structure complexe contenant des attributs qui ne sont pas des valeurs simples mais les listes, des tuples, des listes de tuples, ...

Définir une table **Film** (id, titre, roles, paysSalles) avec 
 - *roles* est une liste de tuples (acteur, nom, rang) où rang est le numéro d'ordre du rôle (attribu *nr_order*)
 - *paysSalles* est la liste des pays où le film est sorti en salles, triée par revenu dans le pays

In [ ]:
%%sql

-- Pour définir la table Film, nous allons procéder en 3 étapes : 
-- 1) création de la table contenant les roles pour chaque id de film
-- 2) création de la table contenant les pays pour chaque id de film
-- 3) création de la table Film en récupérant tous les attributs attendus

-- 1) 
-- On utile collect_list sur plusieurs attributs pour créer des tuples et distinct pour enlever les doublons
create or replace temp view MovieRoles as
select c.movie_id, collect_list(distinct (n.id, r.role,c.nr_order)) as roles
from NameDetail n, Role_Type r, Cast_Info c
where n.id = c.person_id and c.role_id = r.id
group by c.movie_id;

select * from MovieRoles

,movie_id,roles
0,434,"[(1770691, actress, 7), (423423, actor, None),..."
1,1680,"[(1084034, producer, None)]"
2,1841,"[(1053812, actor, None)]"
3,2977,"[(1658620, actor, None), (987079, editor, None..."
4,3228,"[(1744984, actress, 1), (2173288, actress, Non..."


In [ ]:
%%sql

-- 2) 
-- Les pays se trouvent dans info de la table movie_info
create or replace temp view MoviePays as
select c.movie_id,  collect_list(distinct regexp_extract(m.info, '([^:]+)')) as pays
from Cast_Info c, Movie_Info m
where m.movie_id = c.movie_id and m.info_type_id=16
group by c.movie_id;

select * from MoviePays

,movie_id,pays
0,434,[Netherlands]
1,1680,[USA]
2,1841,[Spain]
3,2977,[USA]
4,3228,[USA]


In [ ]:
%%sql

-- 3) 
-- On fait la jointure avec les deux tables précédentes et on récupère les attributs qui nous intéresse
create or replace temp view Film as
select distinct c.movie_id as id, t.title as titre, mr.roles, mp.pays as paysSalles
from Cast_Info c, TitleDetail t, MovieRoles mr, MoviePays mp
where c.movie_id = t.id and c.movie_id = mr.movie_id and c.movie_id = mp.movie_id;

select * from Film -- where size(paysSalles) > 1;

,id,titre,roles,paysSalles
0,434,Brand in Mokum,"[(1770691, actress, 7), (423423, actor, None),...",[Netherlands]
1,1680,A Tiny Problem in the Kitchen,"[(1084034, producer, None)]",[USA]
2,1841,(2005-03-20),"[(1053812, actor, None)]",[Spain]
3,2977,Death: The Gift That Lasts Forever,"[(1658620, actor, None), (987079, editor, None...",[USA]
4,3228,He Followed Me Home/Love 'Em and Flea 'Em,"[(1744984, actress, 1), (2173288, actress, Non...",[USA]


#### Question 7 : Fenêtres
On considère la table **RevenuWeekEndFilm**(id, titre, pays, jour, mois, année, revenu, écrans) dont les informations proviennent de la propriété *weekend gross* de TitleDetail
* un week-end est défini par les attributs (jour, mois, année)
* l'attribut *écrans* correspond à la quantité associée au terme *screens*

Exprimer les requêtes :
 - 7a) Pour chaque mois, le revenu total des films sortis aux USA durant les 3 derniers mois. Le schéma du résultat est : 
     - (mois, année, revenuMoisCourant, revenu3DerniersMois)
 - 7b) Pour chaque pays, le classement des films par revenu total. Le schéma du résultat est :
      - (pays, id, titre, revenuTotal, classement)
 - 7c) Pour chaque film, la date de début et de fin des périodes où les revenus du week end représentent au moins 10% du revenu total annuel du film. Le schéma du résultat est :
     - (id, titre, revenuAnnuel, annee, mois, jour, revenuWeekEnd, numéroPériode)

In [ ]:
%%sql

-- Création de la table RevenuWeekEndFilm
create or replace temp view RevenuWeekEndFilm as
select id, title as titre,
regexp_extract(value, '([^0-9]*)([0-9,m]+) \\(([^\\)]*)\\)', 3) as pays,
regexp_extract(value, '([^0-9]*)([0-9,m]+) \\(([^\\)]*)\\)', 1) as devise,
-- On converti la date en type date
to_date(regexp_extract(value, '([^0-9]*)([0-9,m]+) \\(([^\\)]*)\\) \\(([0-9]{0,2} [a-zA-Z]* [0-9]{0,4})\\)', 4), 'd MMMM yyyy') as date,
-- Pour convertir le revenu en Integer, on enlève d'abord les virgules
cast(regexp_replace(regexp_extract(value, '([^0-9]*)([0-9,m]+) \\(([^\\)]*)\\)', 2),',','') as int) as revenu,
cast(regexp_extract(value, '([^0-9]*)([0-9,m]+) \\(([^\\)]*)\\) \\(([0-9]{0,2}) ([a-zA-Z]*) ([0-9]{0,4})\\)', 4) as int) as jour,
-- Le mois étant en lettres et non en chiffres, on utilise to_date pour convertir en date puis on récupère le mois
-- to_date nous transforme le mois en une date contenant en plus un jour et une année, on récupère seulement le mois
month(to_date(regexp_extract(value, '([^0-9]*)([0-9,m]+) \\(([^\\)]*)\\) \\(([0-9]{0,2}) ([a-zA-Z]*) ([0-9]{0,4})\\)', 5), "MMMM")) as mois,
cast(regexp_extract(value, '([^0-9]*)([0-9,m]+) \\(([^\\)]*)\\) \\(([0-9]{0,2}) ([a-zA-Z]*) ([0-9]{0,4})\\)', 6) as int) as annee,
-- Si il n'y a pas de champ 'screens' alors on met un 0 sinon on cast le nombre en Integer
if (regexp_extract(value, '([^0-9]*)([0-9,m]+) \\(([^\\)]*)\\) \\(([0-9]{0,2}) ([a-zA-Z]*) ([0-9]{0,4})\\) \\(([0-9]+) screen\\)', 7) != '',
cast(regexp_extract(value, '([^0-9]*)([0-9,m]+) \\(([^\\)]*)\\) \\(([0-9]{0,2}) ([a-zA-Z]*) ([0-9]{0,4})\\) \\(([0-9]+) screen\\)', 7) as int), 
0) as ecrans
from TitleDetail
where property = 'weekend gross';

select *
from RevenuWeekEndFilm

,id,titre,pays,devise,date,revenu,jour,mois,annee,ecrans
0,1640952,39 Pounds of Love,USA,$,2006-01-22,90,22,1,2006,1
1,1640952,39 Pounds of Love,USA,$,2006-01-15,1227,15,1,2006,1
2,1640952,39 Pounds of Love,USA,$,2005-12-18,160,18,12,2005,0
3,1640952,39 Pounds of Love,USA,$,2005-12-11,6532,11,12,2005,0
4,1640952,39 Pounds of Love,USA,$,2005-12-04,8055,4,12,2005,1


**7a) Pour chaque mois, le revenu total des films sortis aux USA durant les 3 derniers mois. Le schéma du résultat est :**
- (mois, année, revenuMoisCourant, revenu3DerniersMois)

In [ ]:
%%sql

-- Pour définir la requête 7a), nous allons procéder en 3 étapes : 
-- 1) Calculer le revenu du mois courant
-- 2) Calculer le revenu des 3 derniers mois pour chaque mois
-- 3) Jointure sur les années et les mois pour obtenir le résultat souhaité

-- 1) 
create or replace temp view T_revenuMoisCourant as 
-- On extrait l'annee et le mois de l'attribut date et on somme les revenus pour chaque mois des années
select year(date) as annee, month(date) as mois, sum(revenu) as revenuMoisCourant
from RevenuWeekEndFilm
where pays = 'USA'
group by year(date), month(date)
order by year(date), month(date);

select * from T_revenuMoisCourant;

,annee,mois,revenuMoisCourant
0,1977,10,1540635
1,1982,1,1955016
2,1982,2,1160714
3,1984,2,58689
4,1984,3,1803432


In [ ]:
%%sql

-- 2) 
-- On utilise l'attribut date pour faciliter la fenêtre roulante
create or replace temp view T_revenu3Mois as 
select date, revenu,
sum(revenu) over (order by cast(date as timestamp) RANGE BETWEEN INTERVAL 3 MONTHS PRECEDING AND CURRENT ROW) as revenu3DerniersMois
from RevenuWeekEndFilm
where pays = 'USA'
group by date, revenu
order by date;

select * from T_revenu3Mois

,date,revenu,revenu3DerniersMois
0,1977-10-23,1540635,1540635
1,1982-01-24,237811,237811
2,1982-01-31,1717205,1955016
3,1982-02-07,1160714,3115730
4,1984-02-20,58689,58689


In [ ]:
%%sql

-- 3) Jointure

with T as (
-- On doit récupérer le dernier résultat de chaque mois car la date contient le jour
select year(date) as annee, month(date) as mois, last(revenu3DerniersMois) as revenu3DerniersMois
from T_revenu3Mois
group by year(date), month(date)
)

select t1.mois, t1.annee, revenuMoisCourant, revenu3DerniersMois
from T_revenuMoisCourant t1
left join T on t1.annee = T.annee and t1.mois = T.mois
order by t1.annee, t1.mois

,mois,annee,revenuMoisCourant,revenu3DerniersMois
0,10,1977,1540635,1540635
1,1,1982,1955016,1955016
2,2,1982,1160714,3115730
3,2,1984,58689,58689
4,3,1984,1803432,1862121


**7b) Pour chaque pays, le classement des films par revenu total. Le schéma du résultat est :**
- (pays, id, titre, revenuTotal, classement)

In [ ]:
%%sql
with T as (
-- Calcul du revenu total par film pour chaque pays
select pays, id, titre, sum(revenu) as revenuTotal
from RevenuWeekEndFilm
group by pays, id, titre
order by pays, id, titre, sum(revenu) desc)

-- Ajout et trie du classement
select pays, id, titre, revenuTotal, rank() over (partition by pays order by revenuTotal desc) as classement
from T
order by pays, classement

,pays,id,titre,revenuTotal,classement
0,Argentina,1890399,Final Destination 2,387753,1
1,Argentina,1916458,Ghost Dog: The Way of the Samurai,193407,2
2,Australia,1916458,Ghost Dog: The Way of the Samurai,697828,1
3,Australia,2294518,Slumdog Millionaire,409361,2
4,Austria,1916458,Ghost Dog: The Way of the Samurai,3445077,1


**7c) Pour chaque film, la date de début et de fin des périodes où les revenus du week end représentent au moins 10% du revenu total annuel du film. Le schéma du résultat est :**
- (id, titre, revenuAnnuel, annee, mois, jour, revenuWeekEnd, numéroPériode)

In [ ]:
%%sql
-- Création de la table contenant le revenuAnnuel par film
create or replace temp view T_RevenuAnnuel as 
select id, titre, annee, sum(revenu) as revenuAnnuel
from RevenuWeekEndFilm
group by id, titre, annee
order by id, titre, annee;

select * from T_RevenuAnnuel

,id,titre,annee,revenuAnnuel
0,1635380,102 Dalmatians,2000,43338350
1,1635380,102 Dalmatians,2001,12848606
2,1640952,39 Pounds of Love,2005,18964
3,1640952,39 Pounds of Love,2006,1317
4,1654259,A Nightmare on Elm Street: The Dream Child,1989,20398441


In [ ]:
%%sql
-- Création de la table contenant le revenu d'un week-end par film
create or replace temp view T_RevenuWeekEnd as 
select id, titre, date, sum(revenu) as revenuWeekEnd
from RevenuWeekEndFilm
group by id, titre, date
order by id, titre, date;

create or replace temp view T_Revenu7c as 
select we.id, we.titre, we.date, T_RevenuAnnuel.annee, we.revenuWeekEnd, T_RevenuAnnuel.revenuAnnuel
--,if(we.revenuWeekEnd > T_RevenuAnnuel.revenuAnnuel*0.1, 1, 0) as sup10
from T_RevenuWeekEnd we
left join T_RevenuAnnuel on we.id = T_RevenuAnnuel.id and year(we.date) = annee

-- Week-ends où le revenu représente au moins 10% du revenu total annuel du film
where (we.revenuWeekEnd > T_RevenuAnnuel.revenuAnnuel*0.1)
order by date;

select * from T_Revenu7c

,id,titre,date,annee,revenuWeekEnd,revenuAnnuel
0,2082136,Looking for Mr. Goodbar,1977-10-23,1977,1540635,1540635
1,2477790,Vice Squad,1982-01-31,1982,1717205,3115730
2,2477790,Vice Squad,1982-02-07,1982,1160714,3115730
3,1791664,Crackers,1984-02-20,1984,58689,58689
4,2229370,Racing with the Moon,1984-03-25,1984,1803432,4245660


In [ ]:
%%sql
select id, titre, revenuAnnuel, year(date) as annee, month(date) as mois, day(date) as jour, revenuWeekEnd,
-- On associe chaque date avec un revenu >10% du revenuAnnuel à un nombre, pour chaque film
rank() over (partition by id order by date) as numeroPeriode
from T_Revenu7c

,id,titre,revenuAnnuel,annee,mois,jour,revenuWeekEnd,numeroPeriode
0,1640952,39 Pounds of Love,18964,2005,11,27,4217,1
1,1640952,39 Pounds of Love,18964,2005,12,4,8055,2
2,1640952,39 Pounds of Love,18964,2005,12,11,6532,3
3,1640952,39 Pounds of Love,1317,2006,1,15,1227,4
4,1770926,Chicago Joe and the Showgirl,55179,1990,7,29,14979,1
